In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
df = pd.read_csv("data/patient_information.csv", dtype=str)

In [ ]:
print("number of duplicate rows: ", df[df.duplicated()].shape)
df = df.drop_duplicates()
df = df[['MRN','LOG_ID','HOSP_ADMSN_TIME','HOSP_DISCH_TIME','SURGERY_DATE','IN_OR_DTTM','OUT_OR_DTTM','AN_START_DATETIME','AN_STOP_DATETIME',
         'DISCH_DISP_C', 'DISCH_DISP', 'ICU_ADMIN_FLAG', 'BIRTH_DATE', 'HEIGHT', 'WEIGHT', 'SEX', 'PRIMARY_ANES_TYPE_NM', 'ASA_RATING_C', 'ASA_RATING',
         'PATIENT_CLASS_GROUP', 'PATIENT_CLASS_NM', 'PRIMARY_PROCEDURE_NM']]

In [ ]:
df['MRN'] = df['MRN'].astype("string")
df['LOG_ID'] = df['LOG_ID'].astype("string")
df['HOSP_ADMSN_TIME'] = pd.to_datetime(df['HOSP_ADMSN_TIME'], format="%m/%d/%y %H:%M") 
df['HOSP_DISCH_TIME'] = pd.to_datetime(df['HOSP_DISCH_TIME'], format="%m/%d/%y %H:%M") 
df['SURGERY_DATE'] = pd.to_datetime(df['SURGERY_DATE'], format="%m/%d/%y %H:%M") 
df['IN_OR_DTTM'] = pd.to_datetime(df['IN_OR_DTTM'], format="%m/%d/%y %H:%M")  
df['OUT_OR_DTTM'] = pd.to_datetime(df['OUT_OR_DTTM'], format="%m/%d/%y %H:%M") 
df['AN_START_DATETIME'] = pd.to_datetime(df['AN_START_DATETIME'], format="%m/%d/%y %H:%M")  
df['AN_STOP_DATETIME'] = pd.to_datetime(df['AN_STOP_DATETIME'], format="%m/%d/%y %H:%M")
df['DISCH_DISP'] = df['DISCH_DISP'].astype("string")
df['ICU_ADMIN_FLAG'] = df["ICU_ADMIN_FLAG"].astype("string")
df["HEIGHT"] = df["HEIGHT"].astype("string")
df["SEX"] = df["SEX"].astype("string")
df["PRIMARY_ANES_TYPE_NM"] = df["PRIMARY_ANES_TYPE_NM"].astype("string")
df["ASA_RATING"] = df["ASA_RATING"].astype("string")
df["PATIENT_CLASS_GROUP"] = df["PATIENT_CLASS_GROUP"].astype("string")
df["PATIENT_CLASS_NM"] = df["PATIENT_CLASS_NM"].astype("string")
df["PRIMARY_PROCEDURE_NM"] = df["PRIMARY_PROCEDURE_NM"].astype("string")

Algorithm 1

In [ ]:
def surgery_type(row):
    # Check if all specified columns are null
    if pd.isnull(row[['IN_OR_DTTM', 'OUT_OR_DTTM', 'AN_START_DATETIME', 'AN_STOP_DATETIME']]).all():
        # Check if the difference between admission time and surgery date is greater than one day
        if (row['HOSP_ADMSN_TIME'] - row['SURGERY_DATE']) > dt.timedelta(days=1):
            return 'Surgery Date Passed'
        else:
            return 'Surgery Cancelled'
    else:
        return 'Surgery Performed'

# Apply the function to create the 'SURGERY_TYPE' column
df['SURGERY_TYPE'] = df.apply(surgery_type, axis=1)

Algorithm 2

In [ ]:
def times(p):
  if p['SURGERY_TYPE'] == 'Surgery Date Passed':
    plan_time = p['SURGERY_DATE']
    cancel_time = p['SURGERY_DATE']

  if p['SURGERY_TYPE'] == 'Surgery Performed':
    cancel_time = ''
    if p['SURGERY_DATE'] <= p['HOSP_ADMSN_TIME']:
      plan_time = p['HOSP_ADMSN_TIME'] + dt.timedelta(minutes=1)
    else:
      plan_time = p['SURGERY_DATE']

  if p['SURGERY_TYPE'] == 'Surgery Cancelled':
    plan_time = p['HOSP_ADMSN_TIME'] + dt.timedelta(minutes=1)
    if p['SURGERY_DATE'] + dt.timedelta(hours=23, minutes=59) >= p['HOSP_DISCH_TIME']:
      cancel_time = p['HOSP_DISCH_TIME'] - dt.timedelta(minutes=1)
    else:
      cancel_time = p['SURGERY_DATE'] + dt.timedelta(hours=23, minutes=59)
  return plan_time, cancel_time

df['SRG_PLN_TIME'], df['SRG_CNL_TIME'] = df.apply(times, axis=1, result_type='expand').T.values

In [ ]:
df = df[['MRN','LOG_ID','HOSP_ADMSN_TIME','HOSP_DISCH_TIME','SURGERY_DATE','SURGERY_TYPE','SRG_PLN_TIME','SRG_CNL_TIME',
         'IN_OR_DTTM','OUT_OR_DTTM','AN_START_DATETIME','AN_STOP_DATETIME', 'DISCH_DISP_C', 'DISCH_DISP', 'ICU_ADMIN_FLAG', 
         'BIRTH_DATE', 'HEIGHT', 'WEIGHT', 'SEX', 'PRIMARY_ANES_TYPE_NM', 'ASA_RATING_C', 'ASA_RATING',
         'PATIENT_CLASS_GROUP', 'PATIENT_CLASS_NM', 'PRIMARY_PROCEDURE_NM']]
df = df.sort_values(by=['MRN', 'HOSP_ADMSN_TIME', 'SRG_PLN_TIME', 'LOG_ID'])

In [ ]:
df.to_excel('data/info.xlsx', index=False)